In [9]:
import numpy as np
import pandas as pd
import csv
from scamp import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Softmax,Input
import keras.backend as K 

In [10]:
def creat_model(batch_size,Type):
    
    rnn_units = 12
    n_notes = 30

    input_A = keras.layers.Input(shape=(None,31), name="input_A") # (15, 20, 31)
    input_B = keras.layers.Input(shape=(1,31), name="input_B") # (15, 1, 31)

    hidden1 = keras.layers.LSTM(rnn_units, return_sequences=True)(input_A) # (15,20,12)
    hidden2 = keras.layers.LSTM(rnn_units, return_sequences=True)(hidden1) #(15,20,12)
    
    output_RNN = keras.layers.Dense(n_notes, activation='softmax', name = 'output_RNN')(hidden2) #(15,20,30)

    e = keras.layers.Dense(1, activation='tanh')(hidden2) #(15,20,1)
    e = keras.layers.Reshape([-1])(e) #(15,20)

    alpha = keras.layers.Activation('softmax')(e) #(15,20)
    c = keras.layers.Permute([2, 1])(keras.layers.RepeatVector(rnn_units)(alpha)) #(15,20,12)
    c = keras.layers.Multiply()([hidden2, c]) #(15,20,12)
    c = keras.layers.Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c) #(15,12)

    output_A = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_A')(c) #(15,30) (0, cantus)
    reshape = tf.reshape(output_A,[-1,1,30]) # (15, 1, 30)
    if Type == "_0_1":
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,zero),2) # (15, 1, 31)  (0, cantus)
    elif Type == "_1_0":
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,one),2) # (15, 1, 31)  (0, cantus)

    aux1 = keras.layers.SimpleRNN(rnn_units)(input_B, initial_state=c) #(15,12)
    aux2 = keras.layers.SimpleRNN(rnn_units)(input_output_A, initial_state=c) #(15,12)
    output_B1 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B1')(aux1) #(15,30) (1, counter)
    output_B2 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B2')(aux2) #(15,30) (1, counter)

    model = keras.models.Model([input_A, input_B], [output_RNN, output_A, output_B1, output_B2])
    
    aux_model1 = keras.models.Model([input_A, input_B], output_B1)
    aux_model2 = keras.models.Model(input_A, [output_A, output_B2])
    
    sub_model1 = keras.models.Model(input_A, output_A)
    sub_model2 = keras.models.Model(input_A, output_B2)
    
    att_model = keras.models.Model([input_A, input_B], alpha)
    
    return model,aux_model1,aux_model2,sub_model1,sub_model2,att_model

In [11]:
batch_size = 18

In [12]:
model_0_1_,aux_model1_0_1_,aux_model2_0_1_, sub_model1_0_1_,sub_model2_0_1_,att_model_0_1_ = creat_model(batch_size,'_0_1')
model_1_0_,aux_model1_1_0_,aux_model2_1_0_, sub_model1_1_0_,sub_model2_1_0_,att_model_1_0_ = creat_model(batch_size,'_1_0')

In [168]:
model_0_1_.load_weights('version2_model_0_1_step2_train1.h5') #version 1/2 weights
model_1_0_.load_weights('model_1_0_weight.h5') #version 1/2 weights

In [194]:
model_0_1_.load_weights('version3_model_0_1_step5_train1_random.h5') #version 3 weights
model_1_0_.load_weights('version3_model_1_0_step5_train1_random.h5')

In [19]:
model_0_1_.load_weights('version5_model_0_1_step1_train1.h5') #version 4/5/6/7/8 weights
model_1_0_.load_weights('version5_model_1_0_step1_train1.h5')

In [13]:
df = pd.read_csv('Species.csv', sep=",",index_col = 0)
n = df.shape[0]
X = tf.Variable(np.zeros((2,n*2)),dtype='int32')  # any data tensor
for i in range(0,2*n,2):
    X[0,i].assign(df['Cantus'][i/2] - 43)
    X[1,i].assign(0)
    X[0,i+1].assign(df['Counter'][i/2] - 55)
    X[1,i+1].assign(1)
X

<tf.Variable 'Variable:0' shape=(2, 20000) dtype=int32, numpy=
array([[15, 12, 23, ..., 10, 13,  9],
       [ 0,  1,  0, ...,  1,  0,  1]])>

In [14]:
depth = 30
X = tf.concat([tf.one_hot(X[0,:], depth),tf.dtypes.cast(tf.reshape(X[1,:],[20000,1]),tf.float32)],1)
X

<tf.Tensor: shape=(20000, 31), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices(X[:18000,:])
test_dataset = tf.data.Dataset.from_tensor_slices(X[18000:,:])

In [16]:
n_steps = 20
window_length = n_steps
dataset = train_dataset.window(window_length, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
shuffle_dataset = dataset.shuffle(20000).batch(batch_size,drop_remainder=True)

In [147]:
# version 1/2 test

mode = "max"
#mode = "random"

for window in shuffle_dataset.take(4):
    #print(window)
    X0 = np.zeros((batch_size,10,31))
    X1 = np.zeros((batch_size,10,31))
    
    for g in range(10):
        X0[:,g,:] = window[:,2*g,:]
        X1[:,g,:] = window[:,2*g+1,:]
        
    #print(X0.shape)
    #print(X1.shape)
    
    [pred_o,a] = aux_model2_0_1_.predict(X0) #(18,30)
    pred_observation = np.zeros((batch_size,))
    action = np.zeros((batch_size,))
    my_action = np.zeros((batch_size,30),dtype= 'float32')
    for i in range(batch_size):
        if mode == "max":
            pred_observation[i] = np.where(pred_o[i,:] == np.max(pred_o[i,:]))[0][0]
            action[i] = np.where(a[i,:] == np.max(a[i,:]))[0][0]
        elif mode == "random":
            pred_observation[i] = np.random.choice(30, 1, p=pred_o[i,:])[0]
            action[i] = np.random.choice(30, 1, p=a[i,:])[0]
    
    print('pred_observation:', pred_observation)
    print('action:',action)
    
    one_hot = tf.one_hot(action, 30) #(18,30)
    
    reshape = tf.reshape(one_hot,[batch_size,1,30])
    one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
    input_action = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)
    
    real_o = aux_model1_1_0_.predict((X1, input_action)) # my action on environment; (18,30) real observation
    
    real_observation = np.zeros((batch_size,))
    for i in range(batch_size):
        if mode == "max":
            real_observation[i] = np.where(real_o[i,:] == np.max(real_o[i,:]))[0][0]
        elif mode == "random":
            real_observation[i] = np.random.choice(30, 1, p=real_o[i,:])[0]
    
    print('real_observation:',real_observation)
    print('match_num:',np.where(pred_observation - real_observation == 0)[0].shape[0])
    print('')
    

pred_observation: [27. 27. 27.  8.  8. 27.  9.  9. 27. 27. 27.  8.  8. 27. 27. 27.  8. 27.]
action: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real_observation: [27. 27. 27.  9.  9. 27. 27.  9. 27. 27. 27.  9.  8. 27. 27. 27.  8. 27.]
match_num: 14

pred_observation: [27.  8.  8. 27.  8.  8.  8. 27.  9.  8. 27.  9. 27.  9. 27.  9.  9. 27.]
action: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real_observation: [27.  9.  8. 27.  9.  9.  9. 27.  8.  8. 27.  8. 27.  9. 27.  8.  8. 27.]
match_num: 10

pred_observation: [ 8.  8.  8.  9.  8. 27. 27.  8. 27.  8.  8. 27.  8.  8. 27.  9.  8.  9.]
action: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real_observation: [ 8.  9.  9.  8.  9. 27. 27.  9. 27.  9.  8. 27.  8.  9. 27.  9.  8. 27.]
match_num: 10

pred_observation: [ 8. 27. 27. 27.  9. 27. 27.  8. 27.  8.  8. 27. 27. 27.  9. 27.  8.  8.]
action: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real_observation: [ 8. 27. 27. 27.  8.  8. 27.  8. 27. 

In [169]:
# version 1/2 generate sequence

seq_length = 9

mode = "max"
#mode = "random"

for window in shuffle_dataset.take(1):
    sequence = window
    initial = window
    
    X0 = np.zeros((batch_size,10,31))
    X1 = np.zeros((batch_size,10,31))
    
    action = np.zeros((batch_size,seq_length))
    real_observation = np.zeros((batch_size,seq_length))
    obv_weight = np.zeros((batch_size,seq_length))
    
    for s in range(seq_length):
        for g in range(10):
            X0[:,g,:] = initial[:,2*g,:]
            X1[:,g,:] = initial[:,2*g+1,:]

        [pred_o,a] = aux_model2_0_1_.predict(X0) #(18,30)
        
        for i in range(batch_size):
            if mode == "max":
                action[i,s] = np.where(a[i,:] == np.max(a[i,:]))[0][0]
            elif mode == "random":
                action[i,s] = np.random.choice(30, 1, p=a[i,:])[0]

        
        one_hot = tf.one_hot(action[:,s], 30) #(18,30)

        reshape = tf.reshape(one_hot,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        input_action = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)

        real_o = aux_model1_1_0_.predict((X1, input_action)) # my action on environment; (18,30) real observation

        
        for i in range(batch_size):
            if mode == "max":
                real_observation[i,s] = np.where(real_o[i,:] == np.max(real_o[i,:]))[0][0]
            elif mode == "random":
                real_observation[i,s] = np.random.choice(30, 1, p=real_o[i,:])[0]
            
            index = int(real_observation[i,s])
            obv_weight[i,s] = pred_o[i,index]

        one_hot = tf.one_hot(real_observation[:,s], 30) #(18,30)

        reshape = tf.reshape(one_hot,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        input_observation = tf.concat((reshape,zero),2) # (18, 1, 31)  (1, counter)
        
        sequence = tf.concat((sequence,input_observation,input_action),1)
        
        initial = sequence[:,-20:,:]
        
  
    print('action:',action)
    print('observation:',real_observation)
    print('')
    
    
    step = [];
    for j in range(seq_length):
        step.append("step" + str(j))
    step.append("sequence match")
        
    sample = [];
    for i in range(batch_size):
        sample.append("sequence" + str(i))
    sample.append("step match")
    
    table = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(action[i,j])) + "/" + str(int(real_observation[i,j])) + "/(" + '%.2f'%(obv_weight[i,j]) + ")")
        match = np.sum(obv_weight[i,:])/seq_length
        seq.append('%.2f'%(match))
        table.append(seq)
        
    last_row = [];
    for j in range(seq_length):
        match = np.sum(obv_weight[:,j])/batch_size
        last_row.append('%.2f'%(match))
    total = np.sum(obv_weight)/(seq_length*batch_size)
    last_row.append('%.2f'%(total))
    table.append(last_row)
    
    format_row = "{:>12}" * (seq_length + 2)
    print(format_row.format("act/obv/(pred_obv_weight)", *step))
    for team, row in zip(sample, table):
        print(format_row.format(team, *row))

action: [[24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]
 [24. 24. 24. 24. 24. 24. 24. 24. 24.]]
observation: [[28. 28. 28. 28. 28. 28. 28. 28. 28.]
 [28. 28. 28. 28. 28. 28. 28. 28. 28.]
 [28. 28. 28. 28. 28. 28. 28. 28. 28.]
 [28. 28. 28. 28. 28. 28. 28. 28. 28.]
 [28. 28. 28. 24. 28. 28. 28. 28. 28.]
 [28. 28. 28. 28. 28. 28. 28. 28. 28.]
 [28. 28. 28. 28. 28. 28. 28. 28. 28.]
 [2

In [183]:
sample = 3
music = tf.reshape(sequence[6,:,:],[-1,31])
#music = music[-20:,:]
row,col = music.shape
cantus_lst = [];
counter_lst = [];
for i in range(row):
    clas = np.where(music[i,:-1] == 1)[0][0]
    if music[i,-1] == 0:
        cantus_lst.append(clas+43)
    elif music[i,-1] == 1:
        counter_lst.append(clas+55)

In [184]:
cantus_lst

[54, 53, 54, 51, 53, 52, 49, 47, 48, 50, 55, 66, 70, 48, 49, 48, 54, 60]

In [185]:
counter_lst

[58, 60, 58, 59, 61, 60, 58, 59, 55, 59, 61, 73, 78, 79, 65, 82, 64, 84]

In [186]:
np.array(counter_lst) - np.array(cantus_lst) # real harmonic

array([ 4,  7,  4,  8,  8,  8,  9, 12,  7,  9,  6,  7,  8, 31, 16, 34, 10,
       24], dtype=int64)

In [145]:
s = Session()
s.tempo = 180
piano1 = s.new_part("piano")
piano2 = s.new_part("piano")

def cantus():
    for i in cantus_lst:
        piano2.play_note(i,1,4)
        
def counter():
    for i in counter_lst:
        piano1.play_note(i,1,4)
        
        
s.fast_forward_to_beat(100)

s.start_transcribing()
s.fork(counter)
s.fork(cantus)
s.wait_for_children_to_finish()
performance = s.stop_transcribing()
performance.to_score(title = "First Species Counterpoint", composer = "My programme",time_signature = "4/4").show_xml()

Using preset Piano Merlin for piano
Using preset Piano Merlin for piano


In [188]:
# version 3 test

for window in shuffle_dataset.take(4):
    #print(window)
    X0 = np.zeros((batch_size,10,31))
    X1 = np.zeros((batch_size,10,31))
    
    for g in range(10):
        X0[:,g,:] = window[:,2*g,:]
        X1[:,g,:] = window[:,2*g+1,:] #(18,10,31)
        
    #print(X0.shape)
    #print(X1.shape)
    
    [pred_o,a] = aux_model2_0_1_.predict(X0) #(18,30)
    pred_observation = np.zeros((batch_size,))
    action = np.zeros((batch_size,))
    
    for i in range(batch_size):
        pred_observation[i] = np.where(pred_o[i,:] == np.max(pred_o[i,:]))[0][0]
        action[i] = np.where(a[i,:] == np.max(a[i,:]))[0][0]
    
    one_hot = tf.one_hot(action, 30) #(18,30)
    
    reshape = tf.reshape(one_hot,[batch_size,1,30])
    one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
    input_action = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)
    
    X1 = np.concatenate((X1[:,1:,:],input_action),axis=1)
    
    [pred_a_plus1,o] = aux_model2_1_0_.predict(X1)  
    #(18,30) # environment's prediction about t+1 action based on past actions; the observation the environment choose
    pred_action = np.zeros((batch_size,))
    observation = np.zeros((batch_size,))
    for i in range(batch_size):
        pred_action[i] = np.where(pred_a_plus1[i,:] == np.max(pred_a_plus1[i,:]))[0][0]
        observation[i] = np.where(o[i,:] == np.max(o[i,:]))[0][0]
        
    one_hot = tf.one_hot(observation, 30) #(18,30)
    
    reshape = tf.reshape(one_hot,[batch_size,1,30])
    zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
    input_observation = tf.concat((reshape,zero),2) # (18, 1, 31)  (0, cantus)
    
    X0 = np.concatenate((X0[:,1:,:],input_observation),axis=1)
    
    [pred_o_plus1, a_plus1] = aux_model2_0_1_.predict(X0)
    
    action_plus1 = np.zeros((batch_size,))
    for i in range(batch_size):
        action_plus1[i] = np.where(a_plus1[i,:] == np.max(a_plus1[i,:]))[0][0]
        
        
    print('pred_observation:', pred_observation)
    print('action:',action)
    print('observation:',observation)
    print('match_num:',np.where(pred_observation - observation == 0)[0].shape[0])
    print('')
    
    print('pred_action:', pred_action)
    print('action_plus1:',action_plus1)
    print('match_num:',np.where(pred_action - action_plus1 == 0)[0].shape[0])
    print('')

pred_observation: [12. 12. 12. 22.  5. 22.  5.  5. 12. 12.  5.  5.  5. 22.  5.  5. 12. 22.]
action: [ 9.  9.  9.  9. 24.  9. 24. 24. 24. 25. 24. 24.  0.  9. 24. 24.  9.  9.]
observation: [12. 12. 12. 22. 22. 12.  3.  3. 22. 22.  3.  3.  3. 12. 12.  3. 12. 12.]
match_num: 5

pred_action: [25. 25. 25. 25.  9. 25.  6.  6.  9.  9.  6.  6.  9. 25.  9.  9. 25. 24.]
action_plus1: [24.  9. 25.  9. 24. 24. 24. 24. 24.  9. 24. 24.  0.  9. 24.  0.  9.  9.]
match_num: 2

pred_observation: [22.  5. 12. 27. 12. 22.  5. 12.  8.  5.  5. 12.  5.  5. 22.  5. 12.  5.]
action: [ 9.  0. 25.  9.  9.  9.  0.  9. 24. 24.  0.  9. 24.  0.  9. 24.  9. 24.]
observation: [22.  3. 12. 22. 12. 12. 22. 12.  3. 12.  3. 22.  3.  3. 22.  3. 22. 12.]
match_num: 5

pred_action: [25.  6.  9. 25. 25. 25.  9. 25.  9.  9.  6. 25.  6.  6.  9.  6.  9. 25.]
action_plus1: [ 9.  0.  9.  9.  9.  9. 24.  9. 24. 24.  0.  9.  0.  0.  9. 24.  9. 24.]
match_num: 3

pred_observation: [ 5. 12.  5. 22. 22. 22.  5.  8.  5. 12. 22. 22. 12.  

In [197]:
# version 3 generate sequence

seq_length = 8

mode = "max"
#mode = "random"

for window in shuffle_dataset.take(1):
    sequence = window
    initial = window
    
    X0 = np.zeros((batch_size,10,31))
    X1 = np.zeros((batch_size,10,31))
    
    weight_observation = np.zeros((batch_size,seq_length))
    action = np.zeros((batch_size,seq_length))
    weight_action_plus1 = np.zeros((batch_size,seq_length))
    observation = np.zeros((batch_size,seq_length))
    action_plus1 = np.zeros((batch_size,seq_length))
    
    for s in range(seq_length):
        for g in range(10):
            X0[:,g,:] = initial[:,2*g,:]
            X1[:,g,:] = initial[:,2*g+1,:]

        [pred_o,a] = aux_model2_0_1_.predict(X0) #(18,30)

        for i in range(batch_size):
            if mode == "max":
                action[i,s] = np.where(a[i,:] == np.max(a[i,:]))[0][0]
            elif mode == "random":
                action[i,s] = np.random.choice(30, 1, p=a[i,:])[0]

        one_hot = tf.one_hot(action[:,s], 30) #(18,30)

        reshape = tf.reshape(one_hot,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        input_action = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)

        X1 = np.concatenate((X1[:,1:,:],input_action),axis=1)

        [pred_a_plus1,o] = aux_model2_1_0_.predict(X1)  
        #(18,30) # environment's prediction about t+1 action based on past actions; the observation the environment choose
       
        for i in range(batch_size):
            if mode == "max":                           
                observation[i,s] = np.where(o[i,:] == np.max(o[i,:]))[0][0]
            elif mode == "random":
                observation[i,s] = np.random.choice(30, 1, p=o[i,:])[0]
                
            index = int(observation[i,s])
            weight_observation[i,s] = pred_o[i,index]

        one_hot = tf.one_hot(observation[:,s], 30) #(18,30)

        reshape = tf.reshape(one_hot,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        input_observation = tf.concat((reshape,zero),2) # (18, 1, 31)  (0, cantus)

        X0 = np.concatenate((X0[:,1:,:],input_observation),axis=1)

        [pred_o_plus1, a_plus1] = aux_model2_0_1_.predict(X0)

        for i in range(batch_size):
            if mode == "max":
                action_plus1[i,s] = np.where(a_plus1[i,:] == np.max(a_plus1[i,:]))[0][0]
            elif mode == "random":
                action_plus1[i,s] = np.random.choice(30, 1, p=a_plus1[i,:])[0]
                
            index = int(action_plus1[i,s])
            weight_action_plus1[i,s] = pred_a_plus1[i,index]

        sequence = tf.concat((sequence,input_observation,input_action),1)
        initial = sequence[:,-20:,:]
        
  
    print('action:',action)
    print('observation:',observation)
    print('action_plus1:',action_plus1)
    print('')
    
    
    step = [];
    for j in range(seq_length):
        step.append("step " + str(j))
    step.append("match#(sequence)")
        
    sample = [];
    for i in range(batch_size):
        sample.append("sequence " + str(i))
    sample.append("match#(step)")
    
    table = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(action[i,j])) + "/" + str(int(observation[i,j])) + "/(" + '%.2f'%(weight_observation[i,j]) + ")")
        match = np.sum(weight_observation[i,:])/seq_length
        seq.append('%.2f'%(match))
        table.append(seq)
        
    last_row = [];
    for j in range(seq_length):
        match = np.sum(weight_observation[:,j])/batch_size
        last_row.append('%.2f'%(match))
    total = np.sum(weight_observation)/(seq_length*batch_size)
    last_row.append('%.2f'%(total))
    table.append(last_row)
    
    format_row = "{:>13}" * (seq_length + 2)
    print(format_row.format("act/obv/(pred_obv_weight)", *step))
    for team, row in zip(sample, table):
        print(format_row.format(team, *row))
        
        
    table2 = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(observation[i,j]))  + "/" + str(int(action_plus1[i,j])) + "/(" + '%.2f'%(weight_action_plus1[i,j]) + ")")
        match = np.sum(weight_action_plus1[i,:])/seq_length
        seq.append('%.2f'%(match))
        table2.append(seq)
        
    last_row = [];
    total_num = 0;
    for j in range(seq_length):
        match = np.sum(weight_action_plus1[:,j])/batch_size
        last_row.append('%.2f'%(match))
    total = np.sum(weight_action_plus1)/(seq_length*batch_size)
    last_row.append('%.2f'%(total))
    table2.append(last_row)
    
    format_row = "{:>13}" * (seq_length + 2)
    print(format_row.format("obv/act1//(pred_act1_weight)", *step))
    for team, row in zip(sample, table2):
        print(format_row.format(team, *row))

action: [[24. 24. 24.  0.  0. 24. 18. 18.]
 [ 9.  9.  9.  9. 26.  9.  9.  9.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 2.  2.  9.  9.  2.  2.  0. 24.]
 [ 9.  1.  2.  2.  2.  9.  9.  2.]
 [18. 24.  0.  0.  0.  0.  0.  0.]
 [24. 24.  0.  0.  0.  0.  0.  0.]
 [24. 24. 24. 24. 25. 25.  9. 25.]
 [ 9.  2.  9. 26. 26. 26. 24. 22.]
 [24.  0. 24.  0.  0. 24. 24. 18.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [25. 25. 25. 25. 25. 25. 25.  9.]
 [ 0. 24. 24.  0.  0. 24.  0.  0.]
 [ 9.  2.  2.  2.  2.  2.  2.  0.]
 [17. 22.  9.  9.  9.  9.  9.  1.]
 [24. 24. 24. 24. 24. 18. 18. 25.]]
observation: [[ 3.  3.  3. 22. 22. 25. 25. 25.]
 [21. 22. 22. 22. 22. 12. 22. 22.]
 [ 3.  3.  3.  3.  3.  5.  5.  5.]
 [ 9.  9.  9.  9.  9.  9.  9.  9.]
 [ 3. 28.  3. 22.  9.  9.  9.  9.]
 [ 3.  3.  3.  3.  3.  3.  3.  3.]
 [ 3.  3.  3. 22.  3.  3.  3.  3.]
 [ 3.  3. 28. 22. 22. 22. 25. 22.]
 [ 9.  9.  9.  3. 16. 22. 22. 22.]
 [ 3.  3.  3. 22. 22. 22. 25. 25.

In [60]:
o[0]

array([0.00019319, 0.00045382, 0.00049778, 0.00128009, 0.00102237,
       0.00108642, 0.00249496, 0.00412008, 0.01638836, 0.01986719,
       0.03743871, 0.04368029, 0.08217108, 0.08943228, 0.10772578,
       0.13262376, 0.11122588, 0.09723969, 0.07093284, 0.06245589,
       0.04999767, 0.03354573, 0.0172353 , 0.00568414, 0.00466279,
       0.00252432, 0.00179981, 0.00109694, 0.000349  , 0.00077376],
      dtype=float32)

In [61]:
pred_a_plus1[0]

array([0.00505165, 0.0042974 , 0.00680218, 0.01504538, 0.0197182 ,
       0.0349618 , 0.03080837, 0.04188924, 0.08366255, 0.06824297,
       0.08370838, 0.08113467, 0.08926377, 0.10442644, 0.08149382,
       0.05893774, 0.0362321 , 0.03021151, 0.02717273, 0.02088874,
       0.01934354, 0.01639845, 0.00703552, 0.0053831 , 0.00488435,
       0.00574451, 0.00571032, 0.00547888, 0.00399958, 0.00207211],
      dtype=float32)

In [10]:
# version 4 test

for window in shuffle_dataset.take(4):
    
    initial = window
    
    [pred_o,a] = aux_model2_0_1_.predict(initial) #(18,30)
    pred_observation = np.zeros((batch_size,))
    action = np.zeros((batch_size,))
    for i in range(batch_size):
        pred_observation[i] = np.where(pred_o[i,:] == np.max(pred_o[i,:]))[0][0]
        action[i] = np.where(a[i,:] == np.max(a[i,:]))[0][0]
    
    [pred_a,o] = aux_model2_1_0_.predict(initial) #(18,30)
    pred_action = np.zeros((batch_size,))
    observation = np.zeros((batch_size,))
    for i in range(batch_size):
        pred_action[i] = np.where(pred_a[i,:] == np.max(pred_a[i,:]))[0][0]
        observation[i] = np.where(o[i,:] == np.max(o[i,:]))[0][0]
        
    print('pred_observation:', pred_observation)
    print('observation:',observation)
    print('match_num:',np.where(pred_observation - observation == 0)[0].shape[0])
    print('')
    
    print('action:',action)
    print('pred_action:', pred_action)
    print('match_num:',np.where(pred_action - action == 0)[0].shape[0])
    print('')
    

pred_observation: [19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19.]
observation: [19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19.]
match_num: 18

action: [12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12.]
pred_action: [12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12.]
match_num: 18

pred_observation: [19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19.]
observation: [19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19.]
match_num: 18

action: [12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12.]
pred_action: [12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12.]
match_num: 18

pred_observation: [19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19.]
observation: [19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 19.]
match_num: 18

action: [12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 1

In [29]:
model_0_1_.load_weights('version7_model_0_1_step2_train1_random.h5') #version 4/5/6/7 weights
model_1_0_.load_weights('version7_model_1_0_step2_train1_random.h5')

In [62]:
model_0_1_.save_weights('version9_model_0_1_step5_train1_random.h5') #version 8/9 weights
model_1_0_.load_weights('model_1_0_weight.h5')

In [79]:
model_0_1_.load_weights('model_0_1_weight.h5')
model_1_0_.save_weights('version10_model_1_0_step5_train1_random.h5')

In [80]:
# version 4/5/6/7/8 generate sequence

seq_length = 10

generate_mode = "max"
#generate_mode = "random"

for window in shuffle_dataset.take(1):
    sequence = window
    initial = window
    
    pred_0_1_1 = np.zeros((batch_size,seq_length))
    pred_1_0_0 = np.zeros((batch_size,seq_length))
    
    obv_weight = np.zeros((batch_size,seq_length))
    act_weight = np.zeros((batch_size,seq_length))

    for s in range(seq_length):

        [pred_o,a] = aux_model2_0_1_.predict(initial) #(18,30)

        for i in range(batch_size):
            if generate_mode == "max":
                pred_0_1_1[i,s] = np.where(a[i,:] == np.max(a[i,:]))[0][0]
            elif generate_mode == "random":
                pred_0_1_1[i,s] = np.random.choice(30, 1, p=a[i,:])[0]


        [pred_a,o] = aux_model2_1_0_.predict(initial) #(18,30)
        
        for i in range(batch_size):
            if generate_mode == "max":
                pred_1_0_0[i,s] = np.where(o[i,:] == np.max(o[i,:]))[0][0]
            elif generate_mode == "random":
                pred_1_0_0[i,s] = np.random.choice(30, 1, p=o[i,:])[0]
            
            index = int(pred_1_0_0[i,s])
            obv_weight[i,s] = pred_o[i,index]
            
            index = int(pred_0_1_1[i,s])
            act_weight[i,s] = pred_a[i,index]
            
        
        one_hot = tf.one_hot(pred_0_1_1[:,s], 30) #(18,30)
        reshape = tf.reshape(one_hot,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        out_0_1_1 = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)
        
        one_hot = tf.one_hot(pred_1_0_0[:,s], 30) #(18,30)
        reshape = tf.reshape(one_hot,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        out_1_0_0 = tf.concat((reshape,zero),2) # (18, 1, 31)  (1, counter)

        sequence = tf.concat((sequence,out_1_0_0,out_0_1_1),1)
        initial = sequence[:,-20:,:]
        
    print('action:',pred_0_1_1)
    print('observation:',pred_1_0_0)
    print('')
    
    
    step = [];
    for j in range(seq_length):
        step.append("step" + str(j))
    step.append("sequence match")
        
    sample = [];
    for i in range(batch_size):
        sample.append("sequence" + str(i))
    sample.append("step match")
    
    table = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(pred_0_1_1[i,j])) + "/(" + '%.2f'%(act_weight[i,j]) + ")")
        match_a = np.sum(act_weight[i,:])/seq_length
        seq.append('%.2f'%(match_a))
        table.append(seq)
        
    last_row = [];
    for j in range(seq_length):
        match_a = np.sum(act_weight[:,j])/batch_size
        last_row.append('%.2f'%(match_a))
    total_a = np.sum(act_weight)/(seq_length*batch_size)
    last_row.append('%.2f'%(total_a))
    table.append(last_row)
    
    format_row = "{:>11}" * (seq_length + 2)
    print(format_row.format("act/(pred_act_weight)", *step))
    for team, row in zip(sample, table):
        print(format_row.format(team, *row))

    print('')
        
    table2 = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(pred_1_0_0[i,j])) + "/(" + '%.2f'%(obv_weight[i,j]) + ")")
        match_o = np.sum(obv_weight[i,:])/seq_length
        seq.append('%.2f'%(match_o))
        table2.append(seq)
        
    last_row = [];
    for j in range(seq_length):
        match_o = np.sum(obv_weight[:,j])/batch_size
        last_row.append('%.2f'%(match_o))
    total_o = np.sum(obv_weight)/(seq_length*batch_size)
    last_row.append('%.2f'%(total_o))
    table2.append(last_row)
    
    format_row = "{:>11}" * (seq_length + 2)
    print(format_row.format("obv/(pred_obv_weight)", *step))
    for team, row in zip(sample, table2):
        print(format_row.format(team, *row))

action: [[18. 18. 17. 17. 17. 18. 17. 17. 17. 18.]
 [ 4.  5.  4.  4.  4.  4.  4.  4.  4.  4.]
 [ 5.  7.  5.  4.  4.  5.  5.  5.  5.  4.]
 [ 7.  9.  9.  9.  9.  9.  9. 11. 13. 15.]
 [12. 12. 14. 15. 15. 15. 15. 17. 17. 17.]
 [12. 12. 13. 15. 15. 15. 15. 15. 15. 15.]
 [24. 20. 21. 21. 21. 21. 24. 24. 24. 24.]
 [ 3.  4.  7.  7.  7.  7.  7.  7.  7.  7.]
 [11. 12. 11. 12. 12. 12. 12. 12. 12. 12.]
 [ 4.  5.  4.  5.  5.  4.  5.  4.  4.  4.]
 [20. 21. 24. 24. 24. 24. 24. 24. 24. 24.]
 [20. 21. 24. 24. 24. 24. 24. 24. 24. 24.]
 [ 3.  3.  4.  4.  7.  7.  7.  7.  7.  7.]
 [ 7.  4.  4.  4.  4.  4.  4.  4.  4.  5.]
 [ 7.  9.  9.  9.  9.  9.  9. 12. 12. 13.]
 [ 3.  4.  4.  7.  7.  7.  7.  7.  7.  7.]
 [20. 21. 21. 20. 21. 21. 21. 21. 21. 21.]
 [ 3.  3.  5.  7.  9.  9.  9.  9.  9.  9.]]
observation: [[22. 22. 22. 22. 22. 22. 22. 22. 22. 22.]
 [11. 11. 11. 10.  9.  9.  9.  9.  9. 10.]
 [12. 11. 11. 11. 11. 11. 11. 11. 11. 11.]
 [13. 14. 14. 14. 14. 16. 17. 19. 19. 19.]
 [17. 19. 19. 19. 19. 19. 21. 21